In [11]:

%matplotlib auto
## TODO:
# refactorize to make it model -view- presenter
# pythonize slicing based on return value of spanSelector
# make xLabel of axis2 readable always
# add more info to show
# plot same day rectangles differently!
# update, linewidth, selectionColor on axis2
# adjust colors
# rotate labels,
# fix labelformat
# adjust spacing
# generalize button layout

Using matplotlib backend: Qt5Agg


In [12]:
import os
import inspect
import datetime
from datetime import date, timedelta
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from matplotlib.widgets import Button, SpanSelector
from matplotlib.dates import num2date, DayLocator

In [13]:
class otpParser():
    
    def __init__(self):
        self.headers = ['accountNr', 'T/J', 'sum', 'currency', 'date', 'date2', 'currentValue', 'noIdea',
                   'noIdea2', 'comment', 'noIdea3', 'noIdea4', 'noIdea5']
        
    def parseFiles(self, files):
        dataFrames = [pd.read_csv(file, header=None) for file in files]
        mergedFrame = pd.concat(dataFrames)
        mergedFrame.columns = self.headers                                        
        mergedFrame = mergedFrame.reset_index(drop=True)
        self.mergedFrame = mergedFrame.sort_values(by='date')
        #return self.mergedFrame.head()

    
    def calculateAttributes(self):
        
        self.calculateDates()
        self.separateTransactions()
        self.timeAxis = pd.date_range(self.pandasStartDate, periods=self.timeDelta.days+1, freq='D')
        transactions = {'expense':self.expense,'income':self.income }
        return self.timeAxis, transactions

    def printDf(self):
        return (self.mergedFrame.head())
    
    def calculateDates(self):
        
        start = self.mergedFrame['date'].values[0]
        end = self.mergedFrame['date'].values[-1]
        self.pandasStartDate = "{0}-{1}-{2}".format(str(start)[0:4], str(start)[4:6], str(start)[6:8])
        self.startDate = date(int(str(start)[0:4]), int(str(start)[4:6]), int(str(start)[6:8]))
        self.endDate = date(int(str(end)[0:4]), int(str(end)[4:6]), int(str(end)[6:8]))
        self.timeDelta = self.endDate - self.startDate
        #print (self.startDate, self.endDate, self.timeDelta)
        
    def separateTransactions(self):


        days = [int((self.startDate + timedelta(days=i)).strftime('%Y%m%d')) for i in range(self.timeDelta.days + 1)]
        transDays = self.mergedFrame['date'].values
        heights = self.mergedFrame['sum'].values
        expense = [-figure if figure<0 else 0 for figure in heights]
        income = [figure if figure>0 else 0 for figure in heights]
        Xs = transDays

        values, counts = np.unique(Xs, return_counts=True)
        WholeY=[]
        smallY=np.zeros_like(days)  
        payment = np.zeros_like(days)

        for freq in range(1,max(counts)+1): 
            for val, cnt in zip(values, counts):
                if cnt >= freq:
                    index = np.where(Xs==val)[0][freq-1]
                    if heights[index] > 0:
                        payment [days.index(val)] = heights[index]
                    else:
        #             print (days.index(val), index, val)
                        smallY[days.index(val)] = -heights[index] # maybe transDays
            WholeY.append(smallY)
            smallY=np.zeros_like(days)  


        self.expense = WholeY
        self.income = payment

        pass
        
    def selectMoney(self):
        history = self.mergedFrame['sum']
        
    def createFigures(self):
        self.calculateDates()
        self.separateTransactions()
        self.timeAxis = pd.date_range(self.pandasStartDate, periods=self.timeDelta.days+1, freq='D')
        self.viewer = financeViewer(self.timeAxis,self.charges, self.salary)


In [14]:
class Presenter():
    
    def __init__(self, model, view, files):
        self.model = model
        self.view = view
        self.files = files
        
        self.model.parseFiles(files)
        #self.model.mergeDataFrames()
        self.timeAxis, self.transactions = self.model.calculateAttributes()

#         self.view.connect2presenter(self)
    def showPlots(self):
        self.view.showPlots(self.timeAxis, self.transactions,self.model.mergedFrame)
    
    def showDataframe(self):
        return self.model.mergedFrame
    
    def onResetClicked(self):
        pass
    
    def onSelect(self):
        pass

In [15]:
RED = (0.83921568627450982, 0.15294117647058825, 0.15686274509803921, 1.0)
DARK_RED = (0.49803921568627452, 0.12156862745098039, 0.12156862745098039, 1.0)
GREY = (0.5019607843137255, 0.5450980392156862, 0.5882352941176471, 1)
GREEN = (0.098039215686274508, 0.43529411764705883, 0.23921568627450981, 1.0)
DARK_GREEN = (0.0078431372549019607, 0.25490196078431371, 0.0078431372549019607, 1.0)
PURPLE = '#aaace2' # Color of the buttons, titles
LIGHT_GREEN =  (0.1568627450980392, 0.7058823529411765, 0.38823529411764707, 1)
LIGHT_RED = (1.0, 0.2784313725490196, 0.2784313725490196, 1)

LIGHT_GREY = '#d5d8dc'
width = 0.3
WIDTH = 12
G_RAT = (1 + 5 ** 0.5) / 2 # golden ratio
LABEL_ROTATION = 15 # DEGREES
DATEFORMATSTRING = '%Y-%m-%d'
DATEFROMAT = mdates.DateFormatter(DATEFORMATSTRING)
# to highlight recatangles
dark2light={DARK_RED:LIGHT_RED, DARK_GREEN:LIGHT_GREEN}
# to unhighlight recatangles
dark2normal={DARK_RED:RED, DARK_GREEN:GREEN}
light2normal={LIGHT_RED:RED, LIGHT_GREEN:GREEN}

    
class financeViewer():
    
    def __init__(self):

        self.box = dict(facecolor='blue', pad=3, alpha=0.2, boxstyle="Round4,pad=0.3")
        self.testString ="""Date: {}
                            Sum: {} HUF
                            Comment: {}"""
    
    def connect2presenter(self, presenterObject):
        self.presenter = presenterObject
    
    def createFigure(self):
        print ('{} function is called'.format(inspect.stack()[0][3]))
        self.fig = plt.figure(figsize=(WIDTH, WIDTH/G_RAT),facecolor = LIGHT_GREY)

        self.gsp = gridspec.GridSpec(
                nrows = 3, ncols = 2, wspace = 0.05, hspace = 0.45,
                width_ratios = [G_RAT, 1], height_ratios = [(1+G_RAT)/G_RAT, G_RAT, 1])

        self.ax1 = plt.subplot(self.gsp[0,:])
        self.ax2 = plt.subplot(self.gsp[1:,0])
        self.ax3 = plt.subplot(self.gsp[1,1])
        self.ax4 = plt.subplot(self.gsp[2,1])
        

    def drawAxes(self):

        for ax in [self.ax1,self.ax2,self.ax3, self.ax4]: 
            ax.set_facecolor(GREY)
            
        #####BIG PLOT##       
        self.plotAx1()
        
        ####ZOOM PLOT##
        self.plotAx2()
        
        ##info plot##
        self.txt = self.ax3.text(0.1,0.5,'',
        horizontalalignment='left',
        verticalalignment='center',
        fontsize=12, color='black',
        wrap = True)
        self.ax3.set_xticks([]) 
        self.ax3.set_yticks([]) 
        self.ax3.set_title('info about the transactions', bbox=self.box)

        ### place of buttons##
        self.ax4.set_xticks([]) 
        self.ax4.set_yticks([]) 

    def on_plot_hover(self, event):
#         print ('{} function is called'.format(inspect.stack()[0][3]))
        if not event.inaxes: return
        if event.inaxes!= self.ax2: return

        for idx,bar in enumerate(self.ax2.patches):
            if bar.get_x() < event.xdata < bar.get_x() + bar.get_width():
                if bar.get_y() < event.ydata < bar.get_y() + bar.get_height(): 

                    self.ax2.patches[idx].set_facecolor(dark2light[bar.get_edgecolor()])

                    date_ordinal, y = self.ax2.transData.inverted().transform([event.x, event.y])+0.5
                    
                    # convert the numeric date into a datetime
                    transDate = num2date(date_ordinal).strftime(DATEFORMATSTRING)
                    pdDate = num2date(date_ordinal).strftime('%Y%m%d')

                    try:
                        comment = self.df.loc[(self.df['date'] == int(pdDate)) & (abs(self.df['sum'],)==bar.get_height()),'comment'].iloc[0]
                    except:
                        comment='Record not found'

                    newStr = self.testString.format(transDate,bar.get_height(), comment)
                    self.txt.set_text(newStr)

            else:
                self.ax2.patches[idx].set_facecolor(dark2normal[bar.get_edgecolor()])

        self.fig.canvas.draw()

    def reset_button_on_clicked(self, mouse_event):
        
        self.plotAx2()

    def balanceView_button_on_clicked(self, mouse_event):
        #print ('{} function is not implemented'.format(inspect.stack()[0][3]))
        self.txt.set_text('Not implemented yet')
        pass
    def transView_button_on_clicked(self, mouse_event):
        self.txt.set_text('Not implemented yet')
        pass
    def plotAx2(self, startDate=None, endDate=None):  
        self.ax2.cla()
        self.ax2.set_title('Selected duration', bbox=self.box)
        
        if startDate:
            zoomedTime = self.timeAxis[startDate:endDate]
            zoomedExpense = [expense[startDate:endDate] for expense in self.expense]
            zoomedIncome = self.income[startDate:endDate]
        else:
            zoomedTime = self.timeAxis
            zoomedExpense = self.expense
            zoomedIncome = self.income
        self.ax2.bar(zoomedTime, zoomedIncome, color=GREEN,edgecolor=DARK_GREEN)
        previousBars=copy.deepcopy(zoomedIncome)
        for expense in zoomedExpense:
            self.ax2.bar(zoomedTime, expense, bottom=previousBars,color=RED, edgecolor=DARK_RED)
            previousBars+=expense
                
        if startDate:
            self.ax2.xaxis.set_major_locator(DayLocator())
        self.ax2.xaxis.set_major_formatter(DATEFROMAT)
        plt.setp( self.ax2.xaxis.get_majorticklabels(), rotation=LABEL_ROTATION )
        
    def plotAx1(self):
        self.ax1.set_title('Whole duration',bbox=self.box)

        self.ax1.bar(self.timeAxis, self.income, color=GREEN,edgecolor=DARK_GREEN)
        previousBars=copy.deepcopy(self.income)
        for expense in self.expense:
            self.ax1.bar(self.timeAxis,expense, bottom=previousBars,color=RED, edgecolor=DARK_RED)
            previousBars+=expense

        self.span = SpanSelector(self.ax1, self.onselect, 'horizontal', 
                   rectprops=dict(alpha=0.3, facecolor=RED))
    
        plt.setp( self.ax1.xaxis.get_majorticklabels(), rotation=LABEL_ROTATION )
        self.ax1.xaxis.set_major_formatter(DATEFROMAT)

    def onselect(self, xmin, xmax):

        dayMin, dayMax = sorted((int(xmin-0.5), int(xmax+0.5)))
        ##xmin, xmax is days from zero, if Xaxis is pandas daterange

        yearZero = datetime.datetime.strptime('0001/01/01', "%Y/%m/%d")
        startDate = yearZero + timedelta(days=dayMin)
        endDate = yearZero + timedelta(days=dayMax)
        st=str(startDate)[:10]
        nd=str(endDate)[:10]
        stIdx, = np.where( self.timeAxis.values==np.datetime64(st) )
        endIdx, = np.where( self.timeAxis.values==np.datetime64(nd) )
        stIdx , endIdx = stIdx[0], endIdx[0]

        self.plotAx2(stIdx, endIdx)

        self.fig.canvas.draw()
        
    def makeButtons(self):

        pos = self.ax4.get_position() # get the  position of axis ,which contains the buttons 
        rowNr, colNr = 1,3
        buttonwidth = 0.1
        buttonheight = 0.07
        Vspace = (pos.width - colNr*buttonwidth)/(colNr+1)
        Hspace = (pos.height - rowNr*buttonheight)/(rowNr+1)
        
        reset_button_ax = self.fig.add_axes([pos.x0+Vspace, pos.y0+Hspace, buttonwidth, buttonheight])
        transactionViewButon_ax = self.fig.add_axes([pos.x0+2*Vspace+buttonwidth, pos.y0+Hspace, buttonwidth, buttonheight])
        balanceViewButton_ax = self.fig.add_axes([pos.x0+3*Vspace+2*buttonwidth, pos.y0+Hspace, buttonwidth, buttonheight])

        self.reset_button = Button(reset_button_ax, 'Reset', color=PURPLE, hovercolor=RED)
        self.transactionViewButton = Button(transactionViewButon_ax, 'Transactions', color=PURPLE, hovercolor=RED)
        self.balanceViewButton = Button(balanceViewButton_ax, 'Balance', color=PURPLE, hovercolor=RED)
    
    def connectButtons(self):

        self.reset_button.on_clicked(self.reset_button_on_clicked)
        self.transactionViewButton.on_clicked(self.transView_button_on_clicked)
        self.balanceViewButton.on_clicked(self.balanceView_button_on_clicked)

    
    def showPlots(self, timeAxis, transactions, df):
        self.timeAxis = timeAxis
        self.income = transactions['income']
        self.expense = transactions['expense']
        self.df = df
        self.createFigure()
        self.drawAxes()

        print ('{} function is called'.format(inspect.stack()[0][3]))
        self.fig.canvas.mpl_connect('button_press_event', self.on_plot_hover) 
        self.fig.subplots_adjust(left=0.06, bottom=0.07, right=0.97, top=0.95)
        self.makeButtons()
        self.connectButtons()

        plt.show()

In [16]:
files =  [file for file in os.listdir() if file.lower().endswith('csv')]
model = otpParser()
view = financeViewer()


myApp = Presenter(model, view, files)

In [17]:
myApp.showPlots()

createFigure function is called
showPlots function is called


In [ ]:
# myApp.timeAxis
# myApp.transactions

viewer = financeViewer()
viewer.showPlots(myApp.timeAxis ,myApp.transactions,myApp.showDataframe())


In [ ]:
pdDate = 20170515
figure = 80002
df = myApp.showDataframe()
# comment = df.loc[(df['date'] == pdDate) ,'comment'].iloc[0]
try:
    comment = df.loc[(df['date'] == pdDate) & (abs(df['sum'],)==figure),'comment'].iloc[0]
except:
    comment='Record not found'
print (comment)

In [ ]:
print(myApp.timeAxis)
print (myApp.transactions)

In [ ]:
a=myApp.showDataframe()
df=a

In [ ]:
a.head(2)